In [ ]:
import random
import functools
import itertools
import operator
import gc

with __import__('importnb').Notebook():
    # github.com/deathbeds/importnb
    import eth80
    from eth80 import Eth80Dataset, extract_X_y_train_test
    from feature_extraction import TuckerFeatureExtractor
    from ml_utils import (
        do_every_num_epochs, memreport, train_and_evaluate
    )
    from td_mpr import MPRLowRankCP
    
import matplotlib.pyplot as plt
%matplotlib inline

import tensorly as tl

import torch
import torch.nn as nn
import torch.nn.functional as tnnf


In [ ]:
tl.set_backend("pytorch")
device = torch.device("cuda:0")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
eth80_dataset = Eth80Dataset(
    "/mnt/hdd_1tb/smiles_backup/Documents/datasets/eth80/eth80-cropped-close128/"
)

In [ ]:
def generate_configuration():
    return {
        "extracted_features_shape": (
            random.randint(1, eth80.NUM_ANGLES // 2), # angles mode
            random.randint(1, 3), # channels mode
            random.randint(1, eth80.IMAGE_HEIGHT // 4),
            random.randint(1, eth80.IMAGE_WIDTH // 4)
        ),
        "learning_rate": 10**random.randint(-13, -1),
        "regularization_coefficient": 10**random.randint(-8, 8),
        "polynom_order": random.randint(2, 3),
        "weights_rank": random.randint(1, 100)
    }

It seems the best suitable learning rates decrease as polynom order increases. For polynom order 3 they are around 1e-9, 1e-10. Anything greater than 1e-9 produces NaNs. Anything smaller is too slow.

Some good hyperparameters are listed below

extracted features dimensions [18, 3, 29, 10], lr 1.000000e-09, polynomial order 3, regularization coefficient 1.000000e-03, weights_rank 80

In [ ]:
def make_CP_MPR(
    num_extracted_features, polynom_order, weights_rank,
    optimizer_creator, learning_rate, regularization_coefficient,
    betas=None
):
    model = MPRLowRankCP(
        num_extracted_features, eth80.NUM_CLASSES,
        polynom_order=polynom_order,
        rank=weights_rank,
        bias=True
    )
    
    optimizer_additional_parameters = {}
    if betas is not None:
        optimizer_additional_parameters["betas"] = betas
    
    optimizer = optimizer_creator(
        model.parameters(), lr=learning_rate,
        weight_decay=regularization_coefficient,
        **optimizer_additional_parameters
    )
    return model, optimizer

In [ ]:
def evaluate_CP_MPR():
    extracted_features_shape = [18, 3, 29, 10]
    polynom_order = 3
    weights_rank = 500
    X_train, y_train, X_test, y_test = extract_X_y_train_test(
        eth80_dataset, num_test_objects_per_class=2,
        extracted_features_shape=extracted_features_shape
    )
    model, optimizer = make_CP_MPR(
        functools.reduce(operator.mul, extracted_features_shape),
        polynom_order,
        weights_rank,
        torch.optim.SGD, learning_rate=1e-9,
        regularization_coefficient=1e-3
    )
    train_and_evaluate(
        X_train, y_train, X_test, y_test,
        model, optimizer,
        eval_every_num_epochs=3, plot_every_num_epochs=60,
        num_epochs=130
    )

In [ ]:
evaluate_CP_MPR()